In [ ]:
import os
import requests

# Get PDF document path
pdf_path = "In_human_anat_final.pdf"

In [32]:
import fitz
from tqdm.auto import tqdm

pdf_path = "ln_human_anat_final.pdf"

def text_formatter(text: str) -> str: 
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()

    # Potentially more text formatting functions can go here
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = [] 
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number-10,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_setence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4, # 1 token = ~4 characters
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -10,
  'page_char_count': 333,
  'page_word_count': 84,
  'page_setence_count_raw': 1,
  'page_token_count': 83.25,
  'text': 'LECTURE NOTES    For Nursing Students           Human Anatomy  and Physiology                  Nega Assefa  Alemaya University  Yosief Tsige  Jimma University      In collaboration with the Ethiopia Public Health Training Initiative, The Carter Center,  the Ethiopia Ministry of Health, and the Ethiopia Ministry of Education    2003'},
 {'page_number': -9,
  'page_char_count': 1267,
  'page_word_count': 234,
  'page_setence_count_raw': 9,
  'page_token_count': 316.75,
  'text': 'Funded under USAID Cooperative Agreement No. 663-A-00-00-0358-00.      Produced in collaboration with the Ethiopia Public Health Training Initiative, The Carter  Center, the Ethiopia Ministry of Health, and the Ethiopia Ministry of Education.                  Important Guidelines for Printing and Photocopying  Limited permission is granted free of charge to print or phot

In [33]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()


,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,-10,333,84,1,83.25,LECTURE NOTES For Nursing Students ...
1,-9,1267,234,9,316.75,Funded under USAID Cooperative Agreement No. 6...
2,-8,1093,203,10,273.25,Human Anatomy and Physiology i Preface Th...
3,-7,306,62,3,76.50,Human Anatomy and Physiology ii - Review qu...
4,-6,659,112,5,164.75,Human Anatomy and Physiology iii Acknowledgm...


In [34]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Make sure all sentences are strings (the default type is a spaCy datatype)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/428 [00:00<?, ?it/s]

In [35]:
import random
random.sample(pages_and_texts, k=1)

[{'page_number': 269,
  'page_char_count': 1317,
  'page_word_count': 245,
  'page_setence_count_raw': 18,
  'page_token_count': 329.25,
  'text': 'Human Anatomy and Physiology  270    is an injury to some part of the wall of this system does any  blood escape. On the basis of function, blood vessels may be  classified into three groups:  1. Arteries carry blood from the ventricles (pumping  chambers) of the heart out to the capillaries in organs and  tissue. The smallest arteries are called arterioles.  2. Veins drain capillaries in the tissues and organs and  return the blood to the heart. The smallest veins are the  venules.  3. Capillaries allow for exchanges between the blood and  body cells, or between the blood and air in the lung  tissues. The capillaries connect the arterioles and  venules.    Structure of blood vessels  Arteries, veins and capillaries differ in structure. Three coats  or layers are found in both arteries and veins (Figure 9-7). The  outer most layer is called

In [36]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,page_sentence_count_spacy
count,428.00,428.00,428.00,428.00,428.00,428.00
mean,203.50,972.77,191.65,8.77,243.19,8.70
std,123.70,370.10,66.41,5.35,92.52,5.45
min,-10.00,52.00,13.00,1.00,13.00,1.00
25%,96.75,695.50,151.00,4.00,173.88,4.00
50%,203.50,1067.00,201.00,9.00,266.75,9.00
75%,310.25,1268.25,235.25,12.00,317.06,12.00
max,417.00,1822.00,795.00,31.00,455.50,35.00


In [37]:
num_chunk_list_size = 10

def chunk_text(input_text: list[str], chunk_size: int= num_chunk_list_size) -> list[list[str]]:
    return [input_text[i: i+chunk_size] for i in range(0, len(input_text), chunk_size)]

In [38]:
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = chunk_text(item["sentences"])
    item["sentence_chunks_num"] = len(item["sentence_chunks"])

  0%|          | 0/428 [00:00<?, ?it/s]

In [40]:
rnd = random.sample(pages_and_texts, k=1)
rnd_chunk = rnd[0]["sentence_chunks"]
rnd_chunk


[['Human Anatomy and Physiology  308    Review Questions     1.',
  'What is the purpose of respiration and what are its two  components?',
  ' 2.',
  'Trace the pathway of air from the outside into the blood.',
  ' 3.',
  'What are the advantages of breathing through the nose?',
  ' 4.',
  'Describe the lung cavities.',
  ' 5.',
  'What  muscles  are  used  for  inhalation?'],
 [' Forceful  expiration?',
  ' 6.',
  'What are chemoreceptors and how do they function to  regulate breathing?',
  ' 7.',
  'Define four volumes used to measure breathing.']]

In [41]:
import re

page_chunks = []

for item in tqdm(pages_and_texts):
    for sentence in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_no"] = item["page_number"]

        joined_sentence = "".join(sentence).replace("  "," ").strip()
        joined_sentence = re.sub(r'\.([A-Z])', r'. \1', joined_sentence)

        chunk_dict["chunk_sentences"] = joined_sentence

        chunk_dict["character_count"] = len(joined_sentence)
        chunk_dict["word_count"] = len([word for word in joined_sentence.split(" ")])
        chunk_dict["token_count"] = len(joined_sentence)/4

        page_chunks.append(chunk_dict)

  0%|          | 0/428 [00:00<?, ?it/s]

In [44]:
random.sample(page_chunks, k=1)

[{'page_no': 367,
  'chunk_sentences': 'Human Anatomy and Physiology 368  14.1.1 Male internal reproductive organs  Male internal reproductive organs include the tastes, series of ducts and accessory glands.  14.1.1.1 Tastes   Testes are paired male reproductive organs (gonads), which produce sperm. During fetal life, tests are formed just below the kidneys inside the abdomino-pelvic cavity. By third fetal month it stats is to descend and by the seventh month of fetal life it passes through the inguinal canal. The inguinal canal is a passageway leading to scrotum.  Scrotum is a paunch of skin that hangs between the thighs. The tests complete their descent in to the scrotum shortly before or after birth. Because the tests hang in scrotum out side the body their temperature is of cooler than the body temperature by 3 Degree Fahrenheit. This lower temperature is needed for sperm production and survival. Under the skin of scrotum, Cremaster muscle elevates tests during cold weather.',
  'c

In [47]:
df = pd.DataFrame(page_chunks)
min_token= 30
pages_chunks_over_min_token = df[df["token_count"] > min_token].to_dict(orient="records")
pages_chunks_over_min_token[:2]

[{'page_no': -10,
  'chunk_sentences': 'LECTURE NOTES  For Nursing Students      Human Anatomy and Physiology         Nega Assefa Alemaya University Yosief Tsige Jimma University   In collaboration with the Ethiopia Public Health Training Initiative, The Carter Center, the Ethiopia Ministry of Health, and the Ethiopia Ministry of Education  2003',
  'character_count': 307,
  'word_count': 58,
  'token_count': 76.75},
 {'page_no': -9,
  'chunk_sentences': 'Funded under USAID Cooperative Agreement No.663-A-00-00-0358-00.   Produced in collaboration with the Ethiopia Public Health Training Initiative, The Carter Center, the Ethiopia Ministry of Health, and the Ethiopia Ministry of Education.         Important Guidelines for Printing and Photocopying Limited permission is granted free of charge to print or photocopy all pages of this publication for educational, not-for-profit use by health care workers, students or faculty. All copies must retain all author credits and copyright notices i